## Importing the Data

In [11]:
# Constants
DATASET_DIR = './data/'

import os
import pandas as pd

X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')
y = X['domain1_score']
X = X.dropna(axis=1)
X = X.drop(columns=['rater1_domain1', 'rater2_domain1'])

In [12]:
X.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8


Minimum and Maximum Scores for each essay set.

In [13]:
minimum_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
maximum_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

## Preprocessing the Data

We will preprocess all essays and convert them to feature vectors so that they can be fed into the BiLSTM.

These are all helper functions used to clean the essays.

In [14]:
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model.wv[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

In [15]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/harsha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/harsha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Defining the model 

Here we define a 2-Layer LSTM Model. 

Note that instead of using sigmoid activation in the output layer we will use
Relu since we are not normalising training labels.

In [16]:
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from keras.models import Sequential
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(Bidirectional(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True)))
    model.add(Bidirectional(LSTM(64, recurrent_dropout=0.4)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])

    return model

## Training Phase

Now we train the model on the dataset.

We will use 8-Fold Cross Validation and measure the Quadratic Weighted Kappa for each fold.
We will then calculate Average Kappa for all the folds.

In [17]:
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score

cv = KFold(n_splits = 8, shuffle = True)
results = []
y_pred_list = []
bestRes = -1
count = 1
for traincv, testcv in cv.split(X):
    print(f"\n--------Fold {count}--------")
    X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
    
    train_essays = X_train['essay']
    test_essays = X_test['essay']
    
    sentences = []
    
    #Preprocessing the essays
    for essay in train_essays:
            # Obtaining all sentences from the training essays.
            sentences += essay_to_sentences(essay, remove_stopwords = True)
            
    # Initializing variables for word2vec model.
    num_features = 300 
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3

    print("Training Word2Vec Model...")
    model = Word2Vec(sentences, workers=num_workers, vector_size=num_features, min_count = min_word_count, window = context, sample = downsampling)

    clean_train_essays = []
    
    # Generate training and testing data word vectors.
    for essay_v in train_essays:
        clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
    trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)
    
    clean_test_essays = []
    for essay_v in test_essays:
        clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
    testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )
    
    trainDataVecs = np.array(trainDataVecs)
    testDataVecs = np.array(testDataVecs)

    # Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
    trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
    testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
    
    #Training BiLSTM model
    lstm_model = get_model()
    lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=15)
    lstm_model.summary()
    y_pred = lstm_model.predict(testDataVecs)
    
    
    # Round y_pred to the nearest integer.
    y_pred = np.around(y_pred)
    
    # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
    result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
    print(f"Kappa Score: {result}")
    results.append(result)

    # Save the best model
    if result > bestRes:
        bestRes = result
        model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)
        lstm_model.save('final_model')
    count += 1
        


--------Fold 1--------
Training Word2Vec Model...
Epoch 1/15
178/178 [==============================] - 10s 26ms/step - loss: 37.5662 - mae: 3.3834
Epoch 2/15
178/178 [==============================] - 5s 26ms/step - loss: 16.8868 - mae: 2.2727
Epoch 3/15
178/178 [==============================] - 5s 26ms/step - loss: 10.9291 - mae: 1.8570
Epoch 4/15
178/178 [==============================] - 5s 26ms/step - loss: 9.0685 - mae: 1.6846
Epoch 5/15
178/178 [==============================] - 5s 25ms/step - loss: 8.4629 - mae: 1.6180
Epoch 6/15
178/178 [==============================] - 5s 26ms/step - loss: 7.7521 - mae: 1.5768
Epoch 7/15
178/178 [==============================] - 5s 26ms/step - loss: 7.6293 - mae: 1.5443
Epoch 8/15
178/178 [==============================] - 5s 25ms/step - loss: 6.9988 - mae: 1.4813
Epoch 9/15
178/178 [==============================] - 5s 26ms/step - loss: 7.1847 - mae: 1.4941
Epoch 10/15
178/178 [==============================] - 5s 26ms/step - loss: 6.952

In [20]:
print("Average Kappa score after a 8-fold cross validation: ",np.around(np.array(results).mean(),decimals=4))

Average Kappa score after a 8-fold cross validation:  0.9649


In [19]:
results

[0.9627777697347305,
 0.9616330634471401,
 0.9649158064582236,
 0.968424314134244,
 0.9663560625784211,
 0.9608241090829771,
 0.962290069373062,
 0.9720226185269691]